In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VariableDepthCNN(nn.Module):
    def __init__(self, depth=2, dropout=0.0):
        super(VariableDepthCNN, self).__init__()
        self.depth = depth
        self.dropout = nn.Dropout(dropout)
        self.pool = nn.MaxPool2d(2, 2)

        # Dynamically build convolutional layers
        conv_layers = []
        in_channels = 3
        out_channels_list = [32, 64, 128, 256, 256]  # Up to 5 layers

        for i in range(depth):
            conv_layers.append(nn.Conv2d(in_channels, out_channels_list[i], kernel_size=3, padding=1))
            conv_layers.append(nn.BatchNorm2d(out_channels_list[i]))
            conv_layers.append(nn.ReLU())
            conv_layers.append(self.pool)
            in_channels = out_channels_list[i]

        self.conv = nn.Sequential(*conv_layers)

        # Calculate final feature map size 
        conv_output_size = 32 // (2 ** depth)  # each pool halves the size
        last_channels = out_channels_list[depth - 1]

        self.fc1 = nn.Linear(last_channels * conv_output_size * conv_output_size, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [ ]:
def train_and_evaluate(model, lr, device, epoch, optimizer_name):
    criterion = nn.CrossEntropyLoss()

    # Choose optimizer dynamically
    if optimizer_name.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name.lower() == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_name.lower() == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_name.lower() == 'adamw':
        optimizer = optim.AdamW(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    model.to(device)
    start_time = time.time()

    last_loss = None
    for _ in range(epoch):
        model.train()
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        last_loss = loss.item()

    train_time = time.time() - start_time

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy, train_time, last_loss


In [ ]:
import itertools
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

# Define the search space
learning_rates = [0.01, 0.001, 0.0001]
epochs_list = [5, 10, 15]
depths = [2, 3, 4]
dropouts = [0.0, 0.2, 0.4]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = []

# Fixed parameters
batch_size = 32
optimizer_name = "Adam"
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


# Iterate through all combinations
for lr, epoch, depth, dropout in itertools.product(learning_rates, epochs_list, depths, dropouts):
    model = VariableDepthCNN(depth=depth, dropout=dropout)
    acc, t, loss = train_and_evaluate(model, lr, device, epoch, optimizer_name)
    print(f"[LR={lr}, EPOCHS={epoch}, DEPTH={depth}, DROPOUT={dropout}] → ACC: {acc:.2f}%, Loss: {loss:.4f}, Time: {t:.2f}s")
    results.append((lr, epoch, depth, dropout, acc, loss, t))

# Create and save DataFrame
df = pd.DataFrame(results, columns=["Learning Rate", "Epochs", "Depth", "Dropout", "Test Accuracy", "Train Loss", "Time (s)"])
df.to_csv("grid_search_results.csv", index=False)
print("\n Results saved")